 # EXIOBASE downloand and perparetion script

This script prepares the EXIOBASE3 data for use in the SNAC coupling.

Copyright (C) 2023, XIO Sustainbility Accounting, Inc - All Rights Reserved

Unauthorized copying of this file, via any medium is strictly prohibited
Proprietary and confidential

Written by

    - Richard Wood
    - Konstantin Stadler

EXIOBASE3 is available at:
https://zenodo.org/record/4588235
Note, v3.8.2 is used in the generation of results in this script.

You must set the directory for the exiobase data with
exio3_folder, or you can uncomment text to use the autodownload function

Requires pandas, numpy and pymrio to run.
Pymrio available at: https://github.com/IndEcol/pymrio/


In [ ]:
import pandas as pd
import pymrio
import pymrio.tools.iomath as iomath

Python internal packages (this dont need to be installed, part of standard python)

In [ ]:
from pathlib import Path

## Settings

### Year parameter
First, the years for which the EXIOBASE data should be prepared are defined.
These need to cover the years for the SNAC coupling.

In [ ]:
# years : list[int] = list(range(2012, 2021))
years : list[int] = [2019]

### Locations / folder definitions

First we define the general working directory.

In [ ]:
# Set the work path to the directory where this script is located 
# and if this is not available to the current working directory
try:
    work_path = Path(__file__).parent.absolute()  # when running as script
except NameError:
    work_path = Path.cwd()

data_path: Path = work_path / "data"

Then the file with the characterization factors/exiobase stressor conversin.

In [ ]:
exio_stressor_to_nor = pd.read_excel(
    data_path / "emis_conv_char.xlsx", sheet_name="exio_nor_conv", index_col=0
)
charact_ghg = pd.read_excel(
    data_path / "emis_conv_char.xlsx", sheet_name="nor_char", index_col=0
)

Here we define the folder for the EXIOBASE raw data.
This can be anywhere on the disk (in case EXIOBASE is available locally already).
By default, we set it to ./exiobase within the project repository. This folder
is not tracked by version control (added to .gitignore).
The script works with both, the original
compressed zip downloads or extracted folders.

In [ ]:
exio_raw_path: Path = (
    work_path / "exiobase"
)  

Then the two files containing the additional biogenic CO2 dataset for EXIOBASE

F_CO2_bio_file = data_path / "F_ixi_CO2biogenic.csv"
F_Y_CO2_bio_file = data_path / "F_Y_ixi_CO2biogenic.csv"

Finally we specify the folder for the extracted EXIOBASE data. 
This needs to match the EXIOBASE folder in the main SNAC modelling script.

In [ ]:
exio_prep_path: Path = data_path 
exio_prep_path.mkdir(exist_ok=True)

## EXIOBASE download
NOTE: Comment out this cell if the EXIOBASE files are already downloaded and unpacked.
Update in pymrio coming soon to handle that case as well.

In [ ]:
for year in years:
    print(f"Downloading EXIOBASE for year {year}")

    pymrio.download_exiobase3(
        storage_folder=exio_raw_path,
        doi="10.5281/zenodo.5589597",  # That is version 3.8.2
        system="ixi",
        years=years,
        overwrite_existing=False,
    )

## Emission and import account generation

In [ ]:
yearly_totals = dict()
yearly_imports = dict()
yearly_diag = dict()

for year in years:
    print(f"Processing EXIOBASE files for year {year}")

    exio_files = exio_raw_path.glob(f"IOT_{year}_ixi*")

    if not exio_files:
        print(f"Warning: No EXIOBASE files found for year {year}.")
        continue
    else:
        # We just take the first found, in case we have zip and extracted folders
        exio_file = next(exio_files)

    exio3 = pymrio.parse_exiobase3(exio_file)

    exio3.reset_extensions()

    # Add the biogenic CO2 data
    F_CO2_bio = pd.read_csv(F_CO2_bio_file, index_col=0, header=[0,1], sep='\t').loc[year, :]
    F_CO2_bio.name = 'CO2 – biogenic - air'
    F_CO2_bio = pd.DataFrame(F_CO2_bio).T
    F_CO2_bio.index.name = 'stressor'

    F_Y_CO2_bio = pd.read_csv(F_Y_CO2_bio_file, index_col=0, header=[0,1], sep='\t').loc[year, :]
    F_Y_CO2_bio.name = 'CO2 – biogenic - air'
    F_Y_CO2_bio = pd.DataFrame(F_Y_CO2_bio).T
    F_Y_CO2_bio.index.name = 'stressor'

    F_CO2_bio_unit = pd.DataFrame(index=F_CO2_bio.index, columns=['unit'], data='kg') 

    exio3.satellite = pymrio.concate_extension(exio3.satellite, pymrio.Extension(F=F_CO2_bio,
                                                                                 F_Y=F_Y_CO2_bio,
                                                                                 unit=F_CO2_bio_unit,
                                                                                 name="CO2bio"),
                                               name=exio3.satellite.name)

    # Make a satellite account mirriroing the Norwegian emission data
    emis_like_nor = exio3.satellite.characterize(
        factors=exio_stressor_to_nor.reset_index(),
        characterized_name_column="ghg_type",
        characterization_factors_column="factor",
        characterized_unit_column="ghg_type_unit",
        name="emissions",)

    emis_like_nor.F.index.name = "stressor"

    emis_like_nor.F_Y.index.name = "stressor"

    emis_ghg = emis_like_nor.characterize(
        factors=charact_ghg.reset_index(),
        characterized_name_column="impact",
        characterization_factors_column="factor",
        characterized_unit_column="impact_unit",
        name="ghg",)

    emissions = pymrio.concate_extension(emis_ghg, emis_like_nor, name="emissions")

    exio3.emissions = emissions
    exio3.satellite = None
    exio3.impacts = None

    exio3.calc_all()

    gross_trade = iomath.calc_gross_trade(exio3.Z, exio3.Y)

    imp_emis = ((exio3.emissions.M * gross_trade.bilat_flows["NO"])
                    .T.groupby("sector", sort=False)
                    .sum())

    fd_emis = exio3.emissions.D_cba.loc[:, 'NO'].T.groupby("sector", sort=False).sum()

    imp_monetary = gross_trade.totals.imports['NO']  

    fd_monetary = exio3.Y.loc[:, 'NO'].sum(axis=1).T.groupby("sector", sort=False).sum()
    fd_monetary.name = 'Final demand'

    # Prep output compatible with the matlab script
    sec = pymrio.get_classification('exio3_ixi').sectors
    rename_dict = sec.loc[:, ('ExioName', 'ExioCode')].set_index('ExioName').to_dict()['ExioCode']

    _tot_fd = pd.concat([fd_monetary, fd_emis], axis=1).T
    _tot_fd.loc[:, 'unit'] = exio3.emissions.unit
    _tot_fd.loc['Final demand', 'unit'] = exio3.unit.iloc[0,0]
    total_fd = _tot_fd.set_index('unit', append=True).T.rename(index=rename_dict)
     
    _tot_imp = pd.concat([imp_monetary, imp_emis], axis=1).T
    _tot_imp.loc[:, 'unit'] = exio3.emissions.unit
    _tot_imp.loc['imports', 'unit'] = exio3.unit.iloc[0,0]
    total_imp = _tot_imp.set_index('unit', append=True).T.rename(index=rename_dict)
   
    yearly_totals[year] = total_fd
    yearly_imports[year] = total_imp

    # Diagonalization of the emissions
    diag_coll = dict()

    for emis in exio3.emissions.get_rows():
        print(f"Diagonalizing {emis}")
        eclean = emis.replace(' ', '_') + '_diag'
        setattr(exio3, eclean,  exio3.emissions.diag_stressor(emis, name=emis))
        exio3.calc_all()
        Dcba = getattr(exio3, eclean).D_cba.groupby(level='region').sum().loc[:, 'NO']
        diag_coll[emis] = Dcba.T.rename(index=rename_dict)
        # When short on memory, we can delete the diagonalized emissions again
        delattr(exio3, eclean)

    dd = pd.concat(diag_coll, axis=1, names=['emissions'])
    yearly_diag[year] = dd


    # Q_imp_exio = exio_imp_emis.divide(gross_trade.totals.imports['NO'], axis=0).fillna(0)
    #
    # Q_fd_exio = exio_fd_emis.divide(total_fd, axis=0).fillna(0)

# Saving results
Save the totals and imports in excel, with one sheet per year

In [ ]:
with pd.ExcelWriter(exio_prep_path / 'exio_no_bp_raw.xlsx') as writer:
    for year in years:
        yearly_totals[year].to_excel(writer, sheet_name=f"fd_{str(year)}")
        yearly_imports[year].to_excel(writer, sheet_name=f"imp_{str(year)}")
        yearly_diag[year].to_excel(writer, sheet_name=f"source_{str(year)}")